In [1]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from multiprocess import Pool, cpu_count
import torch
from transformers import pipeline


In [2]:
all_emails = pd.read_csv("all_emails.csv", index_col=0)
all_emails["Date"] = all_emails["Date"].astype("datetime64[ns]")
all_emails["Subject"] = all_emails["Subject"].fillna("")
all_emails["Body_Quoted"] = all_emails["Body_Quoted"].fillna("")
all_emails.dtypes


Top_Level_Folder            object
Mail_Folder                 object
Message_File                 int64
From                        object
To                          object
Cc                          object
Bcc                         object
Date                datetime64[ns]
Subject                     object
Body_Message                object
Body_Quoted                 object
dtype: object

In [3]:
all_emails


,Top_Level_Folder,Mail_Folder,Message_File,From,To,Cc,Bcc,Date,Subject,Body_Message,Body_Quoted
0,taylor-m,all_documents,1,['mark.taylor@enron.com'],['marc.r.cutler@bankamerica.com'],[],[],1998-10-30 15:43:00,Re: Friday,Wish we could go - but we're off to Ft. Lauder...,
1,taylor-m,sent,1,['mark.taylor@enron.com'],['marc.r.cutler@bankamerica.com'],[],[],1998-10-30 15:43:00,Re: Friday,Wish we could go - but we're off to Ft. Lauder...,
2,taylor-m,sent,2,['mark.taylor@enron.com'],['marc.r.cutler@bankamerica.com'],[],[],1998-10-30 15:56:00,Re: Friday,Hey Marc - any chance you guys might like to j...,
3,taylor-m,all_documents,2,['mark.taylor@enron.com'],['marc.r.cutler@bankamerica.com'],[],[],1998-10-30 15:56:00,Re: Friday,Hey Marc - any chance you guys might like to j...,
4,taylor-m,sent,3,['mark.taylor@enron.com'],['shari.stack@enron.com'],[],[],1998-10-30 16:02:00,Petrobras Swap,I think this has already been sent to you. Ju...,
...,...,...,...,...,...,...,...,...,...,...,...
516340,fischer-m,all_documents,428,['denise.williams@enron.com'],['ge_benefits@enron.com'],[],[],2002-07-12 11:13:00,URGENT!!! CUTOVER WEEKEND,It is vital that you leave your computer equip...,
516341,fischer-m,discussion_threads,339,['denise.williams@enron.com'],['ge_benefits@enron.com'],[],[],2002-07-12 11:13:00,URGENT!!! CUTOVER WEEKEND,It is vital that you leave your computer equip...,
516342,fischer-m,notes_inbox,2,['denise.williams@enron.com'],['ge_benefits@enron.com'],[],[],2002-07-12 11:13:00,URGENT!!! CUTOVER WEEKEND,It is vital that you leave your computer equip...,
516343,fischer-m,all_documents,429,['kurt.anderson@enron.com'],['gverkleeren@zilkha.com'],"['rwinsor@zilkha.com', 'jquick@zilkha.com', 'm...","['rwinsor@zilkha.com', 'jquick@zilkha.com', 'm...",2002-07-12 11:36:00,Re: FW: RE: Revised Availability Numbers,"Gary, thank you very much for your feedback. I...","> From: Gary Verkleeren\n> Sent: Friday, July..."


Time for first 10k subjects:

- GPU: 58.7 sec
- CPU: 2 min 45 sec

https://huggingface.co/bhadresh-savani/distilbert-base-uncased-emotion

https://stackoverflow.com/questions/64320883/the-size-of-tensor-a-707-must-match-the-size-of-tensor-b-512-at-non-singleto


In [4]:
# model_name = "bhadresh-savani/distilbert-base-uncased-emotion"
# model_name = "j-hartmann/emotion-english-distilroberta-base"
model_name = "j-hartmann/emotion-english-roberta-large"

classifier = pipeline(
    "text-classification",
    model=model_name,
    top_k=None,
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)


In [5]:
if not os.path.exists("mp_res_df.csv"):
    from model_helpers import make_text_chunks

    if __name__ == "__main__":
        cpus = cpu_count()
        N = all_emails.shape[0]
        # N = 1000
        arglist = list(
            zip(
                [classifier.tokenizer] * N,
                (
                    all_emails["Subject"]
                    + pd.Series(["\n"] * all_emails.shape[0])
                    + all_emails["Body_Message"]
                ).to_list()[0:N],
                [500] * N,
                all_emails.index.to_list()[0:N],
            )
        )
        with Pool(processes=cpus) as mp_pool:
            mp_results = mp_pool.map_async(make_text_chunks, arglist)
            mp_results.wait()
        mp_res_dict = {"message_index": [], "text": []}
        for r in mp_results.get():
            mp_res_dict["message_index"] += r["message_index"]
            mp_res_dict["text"] += r["text"]
        del mp_results
        del mp_pool
        mp_res_df = pd.DataFrame(mp_res_dict)
        del mp_res_dict

    mp_res_df.to_csv("mp_res_df.csv", index=True)

mp_res_df = pd.read_csv("mp_res_df.csv", index_col=0)
mp_res_df["text"] = mp_res_df["text"].fillna("")
mp_res_df


Token indices sequence length is longer than the specified maximum sequence length for this model (1264 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (794 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (685 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (614 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

,message_index,text
0,0,<s>Re: Friday\nWish we could go - but we're of...
1,1,<s>Re: Friday\nWish we could go - but we're of...
2,2,<s>Re: Friday\nHey Marc - any chance you guys ...
3,3,<s>Re: Friday\nHey Marc - any chance you guys ...
4,4,<s>Petrobras Swap\nI think this has already be...
...,...,...
821032,516340,<s>URGENT!!! CUTOVER WEEKEND\nIt is vital that...
821033,516341,<s>URGENT!!! CUTOVER WEEKEND\nIt is vital that...
821034,516342,<s>URGENT!!! CUTOVER WEEKEND\nIt is vital that...
821035,516343,<s>Re: FW: RE: Revised Availability Numbers\nG...


In [6]:
prediction = []
for t in tqdm(mp_res_df["text"].to_list()):
    p = classifier(t)
    prediction.append(p)
prediction[0]


  0%|          | 0/821037 [00:00<?, ?it/s]

/home/florin/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[[{'label': 'sadness', 'score': 0.49911096692085266},
  {'label': 'neutral', 'score': 0.2711654305458069},
  {'label': 'surprise', 'score': 0.12980256974697113},
  {'label': 'joy', 'score': 0.08342836052179337},
  {'label': 'fear', 'score': 0.006577428430318832},
  {'label': 'anger', 'score': 0.006024607922881842},
  {'label': 'disgust', 'score': 0.003890695981681347}]]

In [8]:
n_pred = len(prediction)
all_labels = [l["label"] for l in prediction[0][0]]
scores = {k: [] for k in all_labels}
for p in prediction[0:n_pred]:
    for lsc in p[0]:
        label = lsc["label"]
        score = lsc["score"]
        scores[label].append(score)
scores_df = pd.DataFrame(scores)
scores_df


,sadness,neutral,surprise,joy,fear,anger,disgust
0,0.499111,0.271165,0.129803,0.083428,0.006577,0.006025,0.003891
1,0.499111,0.271165,0.129803,0.083428,0.006577,0.006025,0.003891
2,0.039591,0.300427,0.222579,0.427793,0.004673,0.003807,0.001130
3,0.039591,0.300427,0.222579,0.427793,0.004673,0.003807,0.001130
4,0.103615,0.296843,0.506180,0.041250,0.037420,0.013118,0.001574
...,...,...,...,...,...,...,...
821032,0.489490,0.241759,0.118941,0.053791,0.040572,0.045751,0.009695
821033,0.489490,0.241759,0.118941,0.053791,0.040572,0.045751,0.009695
821034,0.489490,0.241759,0.118941,0.053791,0.040572,0.045751,0.009695
821035,0.514215,0.330147,0.067887,0.036119,0.025373,0.019028,0.007231


In [9]:
mp_res_df.drop(columns=["text"], inplace=True)
mp_res_df = (
    mp_res_df.iloc[0:n_pred, :]
    .join(scores_df)
    .groupby(by=["message_index"])
    .agg("mean")
)
mp_res_df


,sadness,neutral,surprise,joy,fear,anger,disgust
message_index,,,,,,,
0,0.499111,0.271165,0.129803,0.083428,0.006577,0.006025,0.003891
1,0.499111,0.271165,0.129803,0.083428,0.006577,0.006025,0.003891
2,0.039591,0.300427,0.222579,0.427793,0.004673,0.003807,0.001130
3,0.039591,0.300427,0.222579,0.427793,0.004673,0.003807,0.001130
4,0.051528,0.651957,0.207102,0.023535,0.024030,0.027776,0.014073
...,...,...,...,...,...,...,...
516340,0.489490,0.241759,0.118941,0.053791,0.040572,0.045751,0.009695
516341,0.489490,0.241759,0.118941,0.053791,0.040572,0.045751,0.009695
516342,0.489490,0.241759,0.118941,0.053791,0.040572,0.045751,0.009695


In [10]:
all_emails.join(mp_res_df).to_csv("all_emails_scores.csv", index=True)

all_emails = pd.read_csv("all_emails_scores.csv", index_col=0)
all_emails["Date"] = all_emails["Date"].astype("datetime64[ns]")
all_emails["Subject"] = all_emails["Subject"].fillna("")
all_emails["Body_Quoted"] = all_emails["Body_Quoted"].fillna("")
all_emails.dtypes


Top_Level_Folder            object
Mail_Folder                 object
Message_File                 int64
From                        object
To                          object
Cc                          object
Bcc                         object
Date                datetime64[ns]
Subject                     object
Body_Message                object
Body_Quoted                 object
sadness                    float64
neutral                    float64
surprise                   float64
joy                        float64
fear                       float64
anger                      float64
disgust                    float64
dtype: object

In [11]:
from model_helpers import vader_eval

if __name__ == "__main__":
    cpus = cpu_count()
    # N = 100000
    N = all_emails.shape[0]
    arglist = list(
        zip(
            all_emails.index.to_list()[0:N],
            (
                all_emails["Subject"]
                + pd.Series(["\n"] * all_emails.shape[0])
                + all_emails["Body_Message"]
            ).to_list()[0:N],
        )
    )
    with Pool(processes=cpus) as mp_pool:
        mp_results = mp_pool.map_async(vader_eval, arglist)
        mp_results.wait()
    mp_list = [r for r in mp_results.get()]
    del mp_results
    del mp_pool


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [12]:
vader_coef_dict = {}
vader_labels = ["neg", "neu", "pos", "compound", "id"]

for l in vader_labels:
    vader_coef_dict[l] = [c[l] for c in mp_list]
vader_coef = pd.DataFrame(vader_coef_dict)
vader_coef.set_index("id", inplace=True)
vader_coef


,neg,neu,pos,compound
id,,,,
0,0.000,0.936,0.064,0.2144
1,0.000,0.936,0.064,0.2144
2,0.000,0.782,0.218,0.6908
3,0.000,0.782,0.218,0.6908
4,0.013,0.886,0.101,0.9954
...,...,...,...,...
516340,0.019,0.822,0.160,0.8446
516341,0.019,0.822,0.160,0.8446
516342,0.019,0.822,0.160,0.8446


In [13]:
all_emails = all_emails.iloc[0:N, :].join(vader_coef)
all_emails


,Top_Level_Folder,Mail_Folder,Message_File,From,To,Cc,Bcc,Date,Subject,Body_Message,...,neutral,surprise,joy,fear,anger,disgust,neg,neu,pos,compound
0,taylor-m,all_documents,1,['mark.taylor@enron.com'],['marc.r.cutler@bankamerica.com'],[],[],1998-10-30 15:43:00,Re: Friday,Wish we could go - but we're off to Ft. Lauder...,...,0.271165,0.129803,0.083428,0.006577,0.006025,0.003891,0.000,0.936,0.064,0.2144
1,taylor-m,sent,1,['mark.taylor@enron.com'],['marc.r.cutler@bankamerica.com'],[],[],1998-10-30 15:43:00,Re: Friday,Wish we could go - but we're off to Ft. Lauder...,...,0.271165,0.129803,0.083428,0.006577,0.006025,0.003891,0.000,0.936,0.064,0.2144
2,taylor-m,sent,2,['mark.taylor@enron.com'],['marc.r.cutler@bankamerica.com'],[],[],1998-10-30 15:56:00,Re: Friday,Hey Marc - any chance you guys might like to j...,...,0.300427,0.222579,0.427793,0.004673,0.003807,0.001130,0.000,0.782,0.218,0.6908
3,taylor-m,all_documents,2,['mark.taylor@enron.com'],['marc.r.cutler@bankamerica.com'],[],[],1998-10-30 15:56:00,Re: Friday,Hey Marc - any chance you guys might like to j...,...,0.300427,0.222579,0.427793,0.004673,0.003807,0.001130,0.000,0.782,0.218,0.6908
4,taylor-m,sent,3,['mark.taylor@enron.com'],['shari.stack@enron.com'],[],[],1998-10-30 16:02:00,Petrobras Swap,I think this has already been sent to you. Ju...,...,0.651957,0.207102,0.023535,0.024030,0.027776,0.014073,0.013,0.886,0.101,0.9954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516340,fischer-m,all_documents,428,['denise.williams@enron.com'],['ge_benefits@enron.com'],[],[],2002-07-12 11:13:00,URGENT!!! CUTOVER WEEKEND,It is vital that you leave your computer equip...,...,0.241759,0.118941,0.053791,0.040572,0.045751,0.009695,0.019,0.822,0.160,0.8446
516341,fischer-m,discussion_threads,339,['denise.williams@enron.com'],['ge_benefits@enron.com'],[],[],2002-07-12 11:13:00,URGENT!!! CUTOVER WEEKEND,It is vital that you leave your computer equip...,...,0.241759,0.118941,0.053791,0.040572,0.045751,0.009695,0.019,0.822,0.160,0.8446
516342,fischer-m,notes_inbox,2,['denise.williams@enron.com'],['ge_benefits@enron.com'],[],[],2002-07-12 11:13:00,URGENT!!! CUTOVER WEEKEND,It is vital that you leave your computer equip...,...,0.241759,0.118941,0.053791,0.040572,0.045751,0.009695,0.019,0.822,0.160,0.8446
516343,fischer-m,all_documents,429,['kurt.anderson@enron.com'],['gverkleeren@zilkha.com'],"['rwinsor@zilkha.com', 'jquick@zilkha.com', 'm...","['rwinsor@zilkha.com', 'jquick@zilkha.com', 'm...",2002-07-12 11:36:00,Re: FW: RE: Revised Availability Numbers,"Gary, thank you very much for your feedback. I...",...,0.330147,0.067887,0.036119,0.025373,0.019028,0.007231,0.011,0.902,0.087,0.8910


In [14]:
all_emails.drop(
    columns=["Subject", "Body_Message", "Body_Quoted"], inplace=False
).to_csv("sa_results.csv", index=True)
